做的很屎

数据处理还是小白级别 只是用了starter的框架进行处理

对非数数据只会随意转化 没有考虑实际之间的关系

后续学习之后进行更改

当前准确率：Score: 0.16326

In [68]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [70]:
file_path = './input/train.csv'
home_data = pd.read_csv(file_path)

print(home_data.head())

   id  Temparature  Humidity  Moisture Soil Type  Crop Type  Nitrogen  \
0   0           37        70        36    Clayey  Sugarcane        36   
1   1           27        69        65     Sandy    Millets        30   
2   2           29        63        32     Sandy    Millets        24   
3   3           35        62        54     Sandy     Barley        39   
4   4           35        58        43       Red      Paddy        37   

   Potassium  Phosphorous Fertilizer Name  
0          4            5           28-28  
1          6           18           28-28  
2         12           16        17-17-17  
3         12            4        10-26-26  
4          2           16             DAP  


In [72]:
# 得到某一属性的值域
def get_column_range(data, column_name):
    # return list(set(data[column_name]))
    return data[column_name].unique()

# 非数值列 list
nonum_columns_name = ['Soil Type', 'Crop Type', 'Fertilizer Name']

# 查询非数值列属性域
for column_name in nonum_columns_name:
    if column_name in home_data.columns.to_list():
        range_list = get_column_range(home_data, column_name)
        print(column_name, range_list)

Soil_Type_list = ['Red', 'Loamy', 'Sandy', 'Black', 'Clayey']
Crop_Type_list = ['Paddy', 'Sugarcane', 'Barley', 'Millets', 'Tobacco', 'Cotton', 'Wheat', 'Oil seeds', 'Pulses', 'Ground Nuts', 'Maize']
Fertilizer_Name_list = ['DAP', '14-35-14', '28-28', 'Urea', '20-20', '10-26-26', '17-17-17']

Soil Type ['Red', 'Loamy', 'Sandy', 'Black', 'Clayey']
Crop Type ['Paddy', 'Sugarcane', 'Barley', 'Millets', 'Tobacco', 'Cotton', 'Wheat', 'Oil seeds', 'Pulses', 'Ground Nuts', 'Maize']
Fertilizer Name ['DAP', '14-35-14', '28-28', 'Urea', '20-20', '10-26-26', '17-17-17']


In [74]:
# 建立从字符串到数字 和 从数字到字符串的映射
str_to_num_dict = {}
num_to_str_dict = {}

S = 0
for i in range(len(Soil_Type_list)):
    str_to_num_dict[Soil_Type_list[i]] = i
    num_to_str_dict[i] = Soil_Type_list[i]

S = S + len(Soil_Type_list)
for i in range(len(Crop_Type_list)):
    str_to_num_dict[Crop_Type_list[i]] = S + i
    num_to_str_dict[S + i] = Crop_Type_list[i]

S = S + len(Crop_Type_list)
for i in range(len(Fertilizer_Name_list)):
    str_to_num_dict[Fertilizer_Name_list[i]] = S + i
    num_to_str_dict[S + i] = Fertilizer_Name_list[i]

print(str_to_num_dict)
print(num_to_str_dict)


{'Red': 0, 'Loamy': 1, 'Sandy': 2, 'Black': 3, 'Clayey': 4, 'Paddy': 5, 'Sugarcane': 6, 'Barley': 7, 'Millets': 8, 'Tobacco': 9, 'Cotton': 10, 'Wheat': 11, 'Oil seeds': 12, 'Pulses': 13, 'Ground Nuts': 14, 'Maize': 15, 'DAP': 16, '14-35-14': 17, '28-28': 18, 'Urea': 19, '20-20': 20, '10-26-26': 21, '17-17-17': 22}
{0: 'Red', 1: 'Loamy', 2: 'Sandy', 3: 'Black', 4: 'Clayey', 5: 'Paddy', 6: 'Sugarcane', 7: 'Barley', 8: 'Millets', 9: 'Tobacco', 10: 'Cotton', 11: 'Wheat', 12: 'Oil seeds', 13: 'Pulses', 14: 'Ground Nuts', 15: 'Maize', 16: 'DAP', 17: '14-35-14', 18: '28-28', 19: 'Urea', 20: '20-20', 21: '10-26-26', 22: '17-17-17'}


In [76]:
# 数据处理_全部
def process_data(data):
    # 把非数值列的值转为数字
    for column_name in nonum_columns_name:
        if column_name in data.columns:
            data[column_name] = data[column_name].map(str_to_num_dict)
    
    # 数据描述
    data_describe = data.describe()
    print(data_describe)

    # 把数值列的空缺值补为平均值
    for column_name in data.columns:
        data[column_name] = data[column_name].fillna(data_describe[column_name]['mean'])

    return data

# 处理原始数据
home_data = process_data(home_data)

print(home_data.head())

                  id    Temparature       Humidity       Moisture  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean   374999.500000      31.503565      61.038912      45.184147   
std    216506.495284       4.025574       6.647695      11.794594   
min         0.000000      25.000000      50.000000      25.000000   
25%    187499.750000      28.000000      55.000000      35.000000   
50%    374999.500000      32.000000      61.000000      45.000000   
75%    562499.250000      35.000000      67.000000      55.000000   
max    749999.000000      38.000000      72.000000      65.000000   

           Soil Type      Crop Type       Nitrogen      Potassium  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean        2.007555       9.875272      23.093808       9.478296   
std         1.406035       3.191827      11.216125       5.765622   
min         0.000000       5.000000       4.000000       0.000000   
25%         1.000000       7.0000

In [116]:
# 数据划分
y = home_data['Fertilizer Name']

# print(home_data.columns)
features = ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous']
X = home_data[features]

# 划分数据 训练集 + 验证集
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

# 定义随机森林模型
rf_model = RandomForestClassifier(random_state = 1)
rf_model.fit(train_X, train_y)

# 通过验证集求解 MAE平均绝对误差
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print(rf_val_predictions)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

[22 20 22 ... 17 22 18]
Validation MAE for Random Forest Model: 2


In [88]:
# 全训练数据上的模型
rf_model_on_full_data = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 1)
rf_model_on_full_data.fit(X, y)

RandomForestClassifier(max_depth=5, random_state=1)

In [89]:
# 测试数据路径
test_data_path = './input/test.csv'

# 测试数据
test_data = pd.read_csv(test_data_path)

# 数据处理
test_data = process_data(test_data)

# 预测的输入集
test_X = test_data[features]

#预测
test_preds = rf_model_on_full_data.predict(test_X)

                  id    Temparature       Humidity       Moisture  \
count  250000.000000  250000.000000  250000.000000  250000.000000   
mean   874999.500000      31.491648      61.045120      45.190444   
std     72168.927986       4.024093       6.636372      11.793167   
min    750000.000000      25.000000      50.000000      25.000000   
25%    812499.750000      28.000000      55.000000      35.000000   
50%    874999.500000      31.000000      61.000000      45.000000   
75%    937499.250000      35.000000      67.000000      55.000000   
max    999999.000000      38.000000      72.000000      65.000000   

           Soil Type      Crop Type       Nitrogen      Potassium  \
count  250000.000000  250000.000000  250000.000000  250000.000000   
mean        2.010536       9.873888      23.139612       9.487764   
std         1.407273       3.187985      11.215956       5.766860   
min         0.000000       5.000000       4.000000       0.000000   
25%         1.000000       7.0000

In [112]:
# 创建输出 DataFrame
output = pd.DataFrame({
    'id': test_data['id'],
    'Fertilizer Name': [num_to_str_dict[p] for p in test_preds]
})

# 保存结果
output.to_csv('submission.csv', index=False)
print("提交文件已生成！")

提交文件已生成！


In [108]:
list_p = [num_to_str_dict[p] for p in list(set(test_preds))]
print(list_p)
print(list(set(test_preds)))

['14-35-14', '28-28', '20-20', '10-26-26', '17-17-17']
[17, 18, 20, 21, 22]
